# **The Stable Marriage Problem**

Our input will be the same to test all three different algorithms. We will have a dictionary of the men's preferences (men_prefs) and a dictionary of the women's preferences (women_prefs).

In [ ]:
# @title Assign sample data to our dictionaries

# Men's preferences (key: man, value: list of women in order of preference)
men_prefs = {
    'm1': ['w1', 'w2', 'w3'],
    'm2': ['w2', 'w1', 'w3'],
    'm3': ['w1', 'w2', 'w3']
}

# Women's preferences (key: woman, value: list of men in order of preference)
women_prefs = {
    'w1': ['m3', 'm1', 'm2'],
    'w2': ['m1', 'm2', 'm3'],
    'w3': ['m1', 'm2', 'm3']
}

# Print dictionaries
print("Men's Preferences:", men_prefs)
print("Women's Preferences:", women_prefs)

Men's Preferences: {'m1': ['w1', 'w2', 'w3'], 'm2': ['w2', 'w1', 'w3'], 'm3': ['w1', 'w2', 'w3']}
Women's Preferences: {'w1': ['m3', 'm1', 'm2'], 'w2': ['m1', 'm2', 'm3'], 'w3': ['m1', 'm2', 'm3']}


## Gale Shapley Algorithm

Potentially add short Gale-Shapely description and why it is valuable.

**Pseudocode**
```
Initialize all men and women as free.

For each man m: next_proposal_index[m] ← 0   // who to propose to next

While there exists a free man m who still has someone to propose to:
    w ← PrefM[m][ next_proposal_index[m] ]
    next_proposal_index[m] ← next_proposal_index[m] + 1
    If w is free:
        pair m and w
    Else:
        m0 ← current partner of w
        If w prefers m over m0:
            pair m and w
            m0 becomes free
        Else:
            w rejects m (m remains free if he has more to propose)

Return matching
```

In [ ]:
# @title Code for Gale Shapely Algorithm
def gale_shapely(men_prefs, women_prefs):

  # number of men = number of women
  n = len(men_prefs)

  # Keep track of who is free (unengaged)
  free_men = list(men_prefs.keys())

  # Store the current engagements: {woman: man}, starts as an empty dictionary
  engagements = {}

  # Store the next woman each man will propose to
  next_proposal = {m: 0 for m in men_prefs.keys()}

  # While there are still free men
  while free_men:
    # Choose an arbitrary man to be the initializer
    man = free_men[0]

    # Man proposes to next woman on his list
    woman = men_prefs[man][next_proposal[man]]
    next_proposal[man] += 1

    # If the woman is free, they become engaged
    if woman not in engagements:
        engagements[woman] = man
        free_men.remove(man)
    else:
        # Woman is already engaged, check if new man is better
        current_fiance = engagements[woman]

        # Get the preference lists for the woman
        # women_prefs[woman].index(man) gives preference rank of 'man'
        # women_prefs[woman].index(current_fiance) gives preference rank of 'current_fiance'

        if women_prefs[woman].index(man) < women_prefs[woman].index(current_fiance):
            # Woman prefers the new man
            engagements[woman] = man
            free_men.remove(man)
            # The old fiance becomes free again
            free_men.append(current_fiance)
        # Else, woman prefers her current fiance, man remains free and proposes next time

    # Reformat engagements from {woman: man} to {man: woman}
    stable_matching = {v: k for k, v in engagements.items()}

  return stable_matching

In [ ]:
# @title Testing Gale Shapely Algorithm
matching = gale_shapely(men_prefs, women_prefs)
print("\nStable Matching:")
for man, woman in matching.items():
    print(f"{man} is matched with {woman}")


Stable Matching:
m3 is matched with w1
m1 is matched with w2
m2 is matched with w3


## Irving's Algorithm (Stable Roommates Problem) (EDIT THIS!!!)

This algorithm does not take in two preference lists, and instead takes in a single set of even cardinality *n*. Each member in the set ranks all others in order of preference, and a stable matching is a partition of this single set into n/2 pairs of roommates such that no two people who are not roommates both prefer each other to their actual partners. For this algorithm, there exists intances for which no stable matching is possible.

**Pseudocode**
```
"""
Implements Irving's algorithm for the Stable Roommates problem.

Input:
  preferences: dict[str, list[str]]
    Each participant's preference list (complete ranking of all others)

Output:
  matching: dict[str, str] or None
    Stable pairing (symmetric, e.g., {A: B, B: A, ...})
    or None if no stable matching exists.
"""
// Initialize each person as free
For each person p:
    next_proposal_index[p] ← 0   // who to propose to next
    list[p] ← Pref[p]            // copy of preference list

// Phase 1: Proposals
While there exists a person p whose list is non-empty and p has not been semi-engaged:
    q ← list[p][ next_proposal_index[p] ]
    next_proposal_index[p] ← next_proposal_index[p] + 1

    If q is not yet “semi-engaged”:
        pair p and q as temporary partners
    Else:
        r ← current first person in q’s list
        If q prefers p over r:
            Remove all people after p in q’s list
            r becomes free (if needed)
            pair p and q
        Else:
            Remove q from p’s list

// Phase 2: Eliminate rotations
For each person p:
    While there exists a rotation in p’s list:
        Remove the rotation:
            For each person in the rotation:
                Remove the second person in their list
    If any person’s list is empty:
        No stable matching exists
        Return failure

Return matching


```

In [ ]:
# @title Irving's Algorithm (Stable Roommates Problem)
def irving_algorithm(preferences):

    # Make deep copies of preference lists (we’ll modify them)
    prefs = {p: lst.copy() for p, lst in preferences.items()}

    # ---------- Phase 1: Proposal and rejection phase ----------
    free = list(prefs.keys())

    while free:
        person = free.pop(0)
        if not prefs[person]:
            # No one left to propose to → no stable matching possible
            return None

        top_choice = prefs[person][0]

        # If top_choice also prefers 'person' to their current last option, accept tentatively
        if person not in prefs[top_choice]:
            # top_choice has already rejected this person earlier
            prefs[person].remove(top_choice)
            free.append(person)
        else:
            # top_choice considers 'person' and possibly trims worse options
            others = prefs[top_choice]
            index_of_person = others.index(person)
            # Remove all names worse than 'person'
            for rejected in others[index_of_person + 1:]:
                prefs[rejected].remove(top_choice)
            prefs[top_choice] = others[:index_of_person + 1]

    # ---------- Phase 2: Eliminate rotations ----------
    changed = True
    while changed:
        changed = False
        for person in prefs:
            if len(prefs[person]) > 1:
                # Each person points to the first on their list and is pointed to by last
                first = prefs[person][0]
                last = prefs[first][-1]
                if last != person:
                    # Remove each from other's list
                    prefs[person].remove(first)
                    prefs[first].remove(person)
                    changed = True

    # ---------- Check for valid matching ----------
    for p in prefs:
        if len(prefs[p]) != 1:
            return None  # Not stable

    # Create matching dictionary (symmetric)
    matching = {}
    for p in prefs:
        partner = prefs[p][0]
        matching[p] = partner

    return matching


In [ ]:
# @title Testing Irving's Algorithm
roommates_prefs = {
    'A': ['B', 'C', 'D'],
    'B': ['C', 'A', 'D'],
    'C': ['A', 'B', 'D'],
    'D': ['A', 'B', 'C']
}

result = irving_algorithm(roommates_prefs)
if result:
    print("Stable Roommate Matching Found:")
    for p, partner in result.items():
        print(f"{p} ↔ {partner}")
else:
    print("No stable matching exists for this instance.")


No stable matching exists for this instance.


**Sources Used**
---
Irving, Robert W. “An Efficient Algorithm for the ‘Stable Roommates’ Problem.” *Journal of Algorithms*, Volume 6, Issue 4, December 1985, Pages 577-595, An efficient algorithm for the “stable roommates” problem - ScienceDirect. Accessed 12 Nov. 2025.

ChatGPT

## Randomized Algorithms

In [ ]:
# @title Randomized Serial Dictatorship (RSD)
import random

def randomized_serial_dictatorship(men_prefs, women_prefs):
    """
    Fast randomized matching using Random Serial Dictatorship (RSD) style.
    Complexity: O(n log n)
    - Randomizes the order of men
    - Each man picks his top available woman
    - No guarantee of stability
    """
    men = list(men_prefs.keys())
    women = set(women_prefs.keys())  # track available women

    random.shuffle(men)  # random order of proposing men

    matching = {}

    for m in men:
        # pick the first woman in m's preference list who is still available
        for w in men_prefs[m]:
            if w in women:
                matching[m] = w
                women.remove(w)
                break

    return matching


In [ ]:
# @title Testing RSD
matching = randomized_serial_dictatorship(men_prefs, women_prefs)
print("\nRandomized Matching Result:")
for man, woman in matching.items():
    print(f"{man} is matched with {woman}")



Randomized Matching Result:
m2 is matched with w2
m1 is matched with w1
m3 is matched with w3


In [ ]:
# @title Randomized Serial Dictatorship (with heaps)
import random
import heapq

def fast_randomized_matching(men_prefs, women_prefs):
    """
    Fast randomized matching using Random Serial Dictatorship (RSD) approach.
    Complexity: O(n log n)
    - Randomizes the order of men
    - Each man picks his top available woman efficiently using a heap
    - No guarantee of stability
    """
    men = list(men_prefs.keys())
    women = set(women_prefs.keys())  # track available women

    random.shuffle(men)  # random order of proposing men

    # Convert each man's preference list into a heap for fast access
    men_heaps = {}
    for m in men:
        # Assign a random priority to each woman to break ties randomly
        men_heaps[m] = [(i, w) for i, w in enumerate(men_prefs[m])]
        heapq.heapify(men_heaps[m])

    matching = {}

    for m in men:
        while men_heaps[m]:
            _, w = heapq.heappop(men_heaps[m])
            if w in women:
                matching[m] = w
                women.remove(w)
                break

    return matching


In [ ]:
# @title Testing RSD (with heaps)
matching = fast_randomized_matching(men_prefs, women_prefs)
print("\nRandomized Matching Result:")
for man, woman in matching.items():
    print(f"{man} is matched with {woman}")



Randomized Matching Result:
m1 is matched with w1
m2 is matched with w2
m3 is matched with w3
